<a href="https://colab.research.google.com/github/yqwang1/Computational_Neuro/blob/main/NEUR0019_Worksheet_V_Point_Processes_and_Oscillations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The function below generates a Poisson spike train for a simulated neuron with a fixed (homogeneous) rate function.

In [ ]:
# Import the toolboxes we will need
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.signal as sig

# Function to generate homogeneous Poisson spike train of a single simulated neuron
def poiss_spikes(rate=5,duration=10,bin=0.004):
  '''Inputs:
  rate = homogeneous firing rate (Hz)
  duration = duration of the simulated spike train (s)
  bin = time bin (s) - should be small '''

  nbins = np.round(duration/bin+1).astype('int')            # Compute the total number of time bins
  time = np.linspace(start=0,stop=duration,num=nbins)       # Generate a time axis
  spike_train = np.random.poisson(lam=rate*bin,size=nbins)  # Simulate a Poisson spike train
  return spike_train,time                                   # Return those variables

We can now generate a long (two minute) spike train with a given firing rate, and convert the spike count vector to an array of spike times in seconds for subsequent analysis, assuming that the time bins are small enough to ensure that the probability of observing more than one spike in a time bin is very low.

In [ ]:
# Generate long spike train
spks,t = poiss_spikes(rate=5,duration=120,bin=0.004)        # Generate Poisson spike train
mean_r = sum(spks)/max(t)                                   # Compute mean firing rate
print('Mean rate = ' + str(mean_r) + 'Hz')                  # Print to screen

# Convert vector of spike counts to array of spike times
spike_times = t[spks>0]                                     # Find all time bins in which >0 spikes were fired

We can now examine the histogram of inter-spike intervals and temporal auto-correlation for this simulated spike train, and compare that with what we saw from real data in the lectures, and to what we'd expect from a Poisson process

In [ ]:
# Function to plot the ISI histogram
def plot_isis(spike_times,bin_size=0.01,range=1):
  '''Inputs:
  spike_times = array of spike times (s)
  bin_size = time bin for the histogram (s)
  range = window size for the histogram (s)'''

  spike_times = np.sort(spike_times)                                    # Sort spike times into ascending order
  isis = np.diff(spike_times)                                           # Compute ISIs
  plt.figure()                                                          # Generate a new figure
  plt.hist(isis,np.arange(0,range+bin_size,bin_size),density=True)      # Plot a histogram of the inter-spike intervals
  plt.xlabel('Interspike interval (s)',fontsize=18)                     # Label the x-axis
  plt.ylabel('Probability density',fontsize=18)                         # Label the y-axis
  plt.show()                                                            # Display the figure

# Function to plot the temporal auto-correlation
def plot_tac(spike_train,bin_size=0.01,range=1):
  '''Inputs:
  spike_train = vector of spike counts
  bin_size = time bin for the spike count vector (s)
  range = window size for the temporal auto-correlation (s)'''

  autocorr = np.correlate(spike_train,spike_train,mode='full')          # Generate the temporal auto-correlation
  strt_ind = np.size(autocorr)/2 - (range/bin_size)                     # Find the start index of the window of interest
  end_ind  = np.size(autocorr)/2 + (range/bin_size)                     # Find the end index of the window of interest
  autocorr = autocorr[int(strt_ind):int(end_ind+1)]                     # Discard any values beyond the window of time lags we are interested in

  plt.figure()                                                          # Generate a new figure
  plt.plot(np.arange(-range,range+bin_size,bin_size),autocorr)          # Plot the temporal auto-correlation
  plt.xlabel('Time (s)',fontsize=24)                                    # Label the x-axis
  plt.ylabel('Firing rate (Hz)',fontsize=24)                            # Label the y-axis
  plt.show()                                                            # Display the figure

f = plot_isis(spike_times)
print(f)
plot_tac(spks)

**[1]** How do the interspike interval distributions and temporal auto-correlations of real spike trains in the lecture differ from these? What key feature of real biological neurons does the reflect?

**[2]** Edit the ISI histogram  function above so that it also computes and returns the Fano factor of the input spike train. What value would we expect this to take for our simulated spike train?



Now let's look at an example of an inhomogeneous Poisson process - the firing of a simulated place cell. First, we will load some tracking data from a real experiment, and plot that, to visualise the path of the animal over a twenty minute recording session. To do so, upload the 'PosData.csv' and 'EEGData.csv' files to your Google drive by clicking on the folder icon on the left-hand side of the screen in Colab.

In [ ]:
# Load tracking data
pos_data = pd.read_csv('PosData.csv')    # Use the Pandas 'read_csv' function to import tracking data
print(pos_data.head())                   # Print the first five rows of the dataframe

# Plot tracking data
plt.figure()
plt.plot(pos_data['X'],pos_data['Y'])
plt.xlabel('X position')
plt.ylabel('Y position')
plt.show()

Now, before we use this position data (which was recorded at a sample rate of 50Hz, as you can see from the first few lines of data above) to simulate the firing rate of a place cell, we will interpolate to a higher sample rate, to ensure that no more than one spike is fired per time bin

In [ ]:
# Function to resample position data
def resample_pos(pos_data,new_fs):
  '''Inputs:
  pos_data = position data in Pandas format
  new_fs = new sample rate (Hz)
  '''
  x,y,t = pos_data['X'],pos_data['Y'],pos_data['Time']      # Extract x, y and time axis
  old_fs = 1/np.median(np.diff(t))                          # Compute original sample rate (Hz)
  if old_fs != new_fs:                                      # If this is not the same as the new sample rate
    nbins = np.round(max(t)*new_fs).astype('int')           # Compute the total number of new time bins
    new_time = np.linspace(1/new_fs,stop=max(t),num=nbins)  # Generate a new time axis
    x = np.interp(new_time,t,x)                             # Up-sample the x position data to match the EEG sample rate
    y = np.interp(new_time,t,y)                             # Up-sample the y position data to match the EEG sample rate
    t = new_time                                            # Replace original time axis with new time axis
  return x,y,t

# Resample the tracking data to a higher sample rate
x,y,t = resample_pos(pos_data,250)

# Plot tracking data
plt.figure()
plt.plot(x,y)
plt.xlabel('X position')
plt.ylabel('Y position')
plt.show()

Next, we can simulate the activity of a place cell with a 2D Gaussian firing field centred on a particular x and y coordinate. Let's write a function to do so:

In [ ]:
# Generate a Gaussian firing field
def place_field(x,y,t,centre=[60,60],width=5,peak=10):
  '''Inputs:
  x,y,t = x, y position and time axis of tracking data
  centre = centre of the firing field (cm)
  width = width (SD) of the firing field (cm)
  peak = peak firing rate (Hz)
  '''
  dt = np.median(np.diff(t))                                # Time bin (s)
  d = np.sqrt((x-centre[0])**2 + (y-centre[1])**2)          # Distance from place field centre in each time bin
  pf_rate = peak * np.exp(-(d**2) / (2*width)**2)           # Gaussian firing rate
  pf_spikes = np.random.poisson(lam = pf_rate*dt)           # Convert to Poisson spike train
  return pf_rate,pf_spikes

# Inspect the place field Poisson rate function
pf_rate,pf_spikes = place_field(x,y,t)
plt.figure()
plt.scatter(x,y,c=pf_rate,cmap='jet')
plt.xlabel('X position')
plt.ylabel('Y position')
plt.colorbar(label='Firing Rate (Hz)')
plt.show()

**[3]** Plot the ISI histogram and temporal auto-correlation for this cell - how do they differ from above? What does that reflect in the simulated data?


**[4]** Write some code to extract the x and y coordinates of the simulated agent when the simulated place cell fired a spike. Overlay these as red dots on the path plot, to generate a figure that looks like that shown in the lecture slides

Finally, we can incorporate a theta phase preference into the simulated place cell spike train. To do so, we first need to load some LFP data, then filter in the theta band and use the Hilbert transform to extract the phase in each time bin

In [ ]:
# Load EEG data
eeg_data = pd.read_csv('EEGData.csv')                     # Use the Pandas 'read_csv' function to import the EEG data
print(eeg_data.head())

# Function to extract phase of the band-pass filtered signal
def get_phase(eeg,fs,fBand=[6,12]):
  '''Inputs:
  eeg = lfp signal
  fs = sample rate (Hz)
  fBand = frequency band of interest (Hz)
  '''
  b,a = sig.butter(5,fBand,btype='bandpass',fs=fs)        # Generate a Butterworth filter
  filtered = sig.filtfilt(b,a,eeg)                        # Filter the signal (zero-phase)
  analytic = sig.hilbert(filtered)                        # Extract the analytic signal (Hilbert transform)
  phase = np.angle(analytic)                              # Extract the phase
  return filtered,phase

# Extract theta phase of the EEG data
fs = 1 / np.median(np.diff(eeg_data['Time']))             # Extract the sample rate (Hz)
eeg_filt,eeg_phs = get_phase(eeg_data['EEG'],fs)          # Extract phase and filtered signal

# Plot to visually confirm that the filtering has worked as expected
plt.figure()
plt.subplot(2,1,1)
plt.plot(eeg_data['Time'],eeg_data['EEG'],label='Raw')
plt.plot(eeg_data['Time'],eeg_filt,color='r',label='Filtered')
plt.ylabel('Amplitude (uV)')
plt.xlim((10,12))
plt.legend()
plt.subplot(2,1,2)
plt.plot(eeg_data['Time'],eeg_phs)
plt.xlabel('Time (s)')
plt.ylabel('Phase (rad)')
plt.xlim((10,12))
plt.show()

We can now specify a new function to simulate the activity of a place cell with a 2D Gaussian firing field and preferred firing phase, using a von Mises distribution

In [ ]:
# Generate a Gaussian firing field
def place_field(x,y,t,eeg_phs,centre=[60,60],width=5,peak=10,kappa=1,phase=0):
  '''Inputs:
  centre = centre of the firing field (cm)
  width = width (SD) of the firing field (cm)
  peak = peak firing rate (Hz)
  kappa = depth of phase modulation (von Mises kappa)
  phase = preferred firing phase (von Mises phi)
  '''
  dt = np.median(np.diff(t))                                                # Time bin (s)
  d = np.sqrt((x-centre[0])**2 + (y-centre[1])**2)                          # Distance from place field centre in each time bin
  pf_rate = peak * np.exp(-(d**2) / (2*width)**2)                           # Gaussian firing rate
  pf_phase 	= np.exp(kappa*np.cos(phase-eeg_phs))                           # von Mises phase preference
  pf_spikes = np.random.poisson(lam = pf_rate*pf_phase*dt)                  # Convert to Poisson spike train
  return pf_rate,pf_spikes

# Inspect the place field Poisson rate function
pf_rate,pf_spikes = place_field(x,y,t,eeg_phs)
plt.figure()
plt.scatter(x,y,c=pf_rate,cmap='jet')
plt.xlabel('X position')
plt.ylabel('Y position')
plt.colorbar(label='Firing Rate (Hz)')
plt.show()

**[5]** Compute the circular mean (i.e preferred firing phase) and resultant vector length (i.e. measure of phase locking) for the simulated spike train

**[6]** Use a shuffling procedure to establish whether the cell is significantly phase locked.

Finally - if you wish - try adjusting the parameters used to simulate this  spike train and see how that affects the output.

If you fancy a harder challenge, then you could also try to adapt the code above so that the cell exhibits phase precession. To do so, you may wish to make use of the following function, which computes the relative distance travelled across the place field in the current direction of motion.